In [3]:
import pandas as pd
import numpy as np
import shapely as sh
import fiona
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
import json
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
bos_311_file = pd.read_csv('/Users/DavidStier/Documents/Python and Data Science/DSI Class/Final Project/Datasets/Audit Trail Datasets/BOS_311_FOR_MODEL(from sql).csv')

In [5]:
bos_311_file.shape

(816385, 16)

In [6]:
bos_311_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816385 entries, 0 to 816384
Data columns (total 16 columns):
CASE_ENQUIRY_ID         816385 non-null int64
OPEN_DT                 816385 non-null object
REASON                  816385 non-null object
TYPE                    816385 non-null object
land_usage              612027 non-null object
Property_Type           790597 non-null object
Property_ID             790597 non-null float64
LOCATION_STREET_NAME    790597 non-null object
Location                790603 non-null object
LATITUDE                816385 non-null float64
LONGITUDE               816385 non-null float64
Geocoded_Location       816385 non-null object
TYPE:1                  816385 non-null object
REASON:1                816385 non-null object
Total_count             816385 non-null int64
Model_category          816385 non-null object
dtypes: float64(3), int64(2), object(11)
memory usage: 99.7+ MB


In [7]:
bos_311_file_4nulls = bos_311_file
bos_311_file_4nulls = bos_311_file_4nulls[pd.isnull(bos_311_file_4nulls['Property_ID'])]


In [8]:
bos_311_file_4nulls.shape

(25788, 16)

In [9]:
bos_311_file_4nulls['TYPE:1'].unique()

array(['Street Light Outages', 'Sign Repair', 'Tree in Park',
       'General Lighting Request',
       'General Comments For a Program or Policy', 'Graffiti Removal',
       'Park Improvement Requests', 'Schedule a Bulk Item Pickup',
       'Tree Emergencies', 'Street Light Outages (Internal)',
       'Request for Litter Basket Installation',
       'Pavement Marking Maintenance', 'Sewage/Septic Back-Up',
       'Parks Lighting/Electrical Issues', 'Request for Pothole Repair',
       'Rodent Activity', 'Park Maintenance Requests', 'Overcrowding',
       'Major System Failure', 'General Comments For An Employee',
       'Requests for Street Cleaning', 'Sidewalk Repair (Make Safe)',
       'Bed Bugs', 'Tree Maintenance Requests', 'Pigeon Infestation',
       'Trash on Vacant Lot', 'Contractor Complaints',
       'Highway Maintenance', 'Public Works General Request',
       'Street Light Knock Downs', 'Missing Sign',
       'Missed Trash/Recycling/Yard Waste/Bulk Item', 'Roadway Repair',

In [10]:
# Create separate file for all non-null rows - but do not exclude where 'land_usage' is the only null value in the row

In [11]:
bos_311_file_noNulls = bos_311_file.dropna(subset=['Property_ID',], how='all')

bos_311_file_noNulls.shape

(790597, 16)

In [12]:
bos_311 = bos_311_file_noNulls

In [13]:
bos_311.describe()

,CASE_ENQUIRY_ID,Property_ID,LATITUDE,LONGITUDE,Total_count
count,7.905970e+05,790597.000000,790597.000000,790597.000000,790597.000000
mean,1.010011e+11,103562.229777,42.323935,-71.085938,5914.673807
std,4.966099e+05,98362.348785,0.035064,0.036648,8284.725778
min,1.010003e+11,1.000000,42.230200,-71.185400,1.000000
25%,1.010008e+11,17610.000000,42.293000,-71.112400,773.000000
50%,1.010012e+11,84168.000000,42.329700,-71.076700,3251.000000
75%,1.010016e+11,146395.000000,42.353200,-71.058700,6321.000000
max,1.010019e+11,419843.000000,42.395200,-70.994900,28859.000000


In [14]:
bos_311['OPEN_DT'][0][6:10]

'2013'

In [15]:
bos_311['311_year'] = bos_311['OPEN_DT'].apply(lambda x: int(x[6:10]))

/Applications/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [16]:
bos_311.shape
import pandas as pd

In [17]:
addresses = pd.read_csv('/Users/DavidStier/Documents/Python and Data Science/DSI Class/Final Project/Datasets/Audit Trail Datasets/Boston Master_Address_List (url t85d-b449).csv')

In [18]:
bos_addr_jointable = addresses[['Property_ID', 'parcel_num']]

In [19]:
bos_addr_jointable.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331447 entries, 0 to 331446
Data columns (total 2 columns):
Property_ID    331447 non-null int64
parcel_num     331447 non-null int64
dtypes: int64(2)
memory usage: 5.1 MB


In [20]:
type(bos_311['Property_ID'][0])

numpy.float64

In [21]:
bos_311['Property_ID'] = bos_311['Property_ID'].astype(int)

/Applications/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [22]:
type(bos_311['Property_ID'][0])

numpy.int64

In [24]:
bos_311.shape

(790597, 17)

In [26]:
bos311_2 = pd.merge(bos_311, bos_addr_jointable, left_on = 'Property_ID', right_on = 'Property_ID', how = 'left')
bos311_2.shape

(790597, 18)

In [28]:
b311_groupby = bos311_2.groupby(['parcel_num','311_year','Model_category'],as_index = False).size()

In [29]:
b311_groupby.head()

parcel_num   311_year  Model_category
100001000.0  2012      Building          1
                       Enviro            1
             2013      Graffiti          1
             2016      Enviro            1
100002000.0  2012      Building          1
dtype: int64

In [30]:
len(b311_groupby)

438993

In [28]:
b311_groupby[0:20].to_csv('/Users/DavidStier/Downloads/b311grbytest.csv')

In [29]:
b311_groupby[0:20]

Property_ID  311_year  Model_category 
1.0          2011      Street-Sidewalk    2
             2012      Enviro             1
                       excluded           4
             2013      Enviro             1
                       Street-Sidewalk    3
                       excluded           1
             2014      Street-Sidewalk    3
             2015      Enviro             1
                       excluded           5
2.0          2013      Street-Sidewalk    1
             2014      Street-Sidewalk    2
                       excluded           1
             2015      Street-Sidewalk    1
             2016      Street-Sidewalk    1
3.0          2011      Street-Sidewalk    2
                       excluded           5
             2012      Enviro             1
                       Street-Sidewalk    6
                       excluded           5
             2013      Street-Sidewalk    2
dtype: int64

In [30]:
# Would actually like to get just 1 Parcel ID per row, and then columns for the year and commplaint type counts

In [33]:
b311_gb2 = pd.pivot_table(bos311_2, index=['parcel_num'], columns=['311_year','Model_category'], aggfunc=len, fill_value=0)


In [34]:
b311_gb2.tail(5)

CASE_ENQUIRY_ID                                           \
311_year                  2011                                            
Model_category        Building Enviro Graffiti Housing Recycling-Pickup   
parcel_num                                                                
2.205664e+09                 0      0        0       0                1   
2.205665e+09                 0      0        0       0                0   
2.205665e+09                 0      0        0       0                0   
2.205668e+09                 0      0        0       0                0   
2.205669e+09                 0      0        0       0                0   

                                                                ...     \
311_year                                          2012          ...      
Model_category Street-Sidewalk Trees excluded Building Enviro   ...      
parcel_num                                                      ...      
2.205664e+09                 0     0        0        0      0   ...      
2.205665e+09                 0     0        1        0      0   ...      
2.205665e+09                 0     0        0        0      0   ...      
2.205668e+09                 0     0        0        0      0   ...      
2.205669e+09                 0     0        1        0      0   ...      

               Total_count                                            \
311_year              2015              2016                           
Model_category       Trees excluded Building Enviro Graffiti Housing   
parcel_num                                                             
2.205664e+09             0        0        0      0        0       0   
2.205665e+09             0        0        0      0        0       0   
2.205665e+09             0        1        0      0        0       0   
2.205668e+09             0        0        0      1        0       0   
2.205669e+09             0        2        1      0        0       0   

                                                                
311_year                                                        
Model_category Recycling-Pickup Street-Sidewalk Trees excluded  
parcel_num                                                      
2.205664e+09                  0               0     0        0  
2.205665e+09                  0               0     0        0  
2.205665e+09                  0               0     0        0  
2.205668e+09                  0               1     0        1  
2.205669e+09                  0               2     0        0  

[5 rows x 720 columns]

In [37]:
b311_gb2.shape

(100489, 720)

In [36]:
b311_gb2.to_csv('/Users/DavidStier/Downloads/GB311bigtest2.csv')

###  Made slight edits to file in excel : removed multi-index, changed col. names for patsy, removed 'duplicate' count data (because data was only 'count' of calls the individual column counted didn't matter

In [58]:
b311_gb2_backin = pd.read_csv('/Users/DavidStier/Documents/Python and Data Science/DSI Class/Final Project/Datasets/Audit Trail Datasets/GB311bigtest(reduced).csv')

In [60]:
b311_gb2_backin[0:5][0:25]

,Property_ID,Thr12011to2013_Building,Thr12011to2013_Enviro,Thr12011to2013_Graffiti,Thr12011to2013_Housing,Thr12011to2013_Recycling-Pickup,Thr12011to2013_Street-Sidewalk,Thr12011to2013_Trees,Thr12011to2013_excluded,Thr12014to2015_Building,...,Thr12015_Trees,Thr12015_excluded,Thr12016_Building,Thr12016_Enviro,Thr12016_Graffiti,Thr12016_Housing,Thr12016_Recycling-Pickup,Thr12016_Street-Sidewalk,Thr12016_Trees,Thr12016_excluded
0,1,0,2,0,0,0,5,0,5,0,...,0,5,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,0,1,0,0,0,10,0,11,0,...,0,6,0,0,0,0,0,0,0,0
3,4,0,1,0,0,0,1,0,1,0,...,0,4,0,0,0,0,0,0,0,0
4,5,0,9,0,0,0,4,0,19,1,...,0,3,0,0,0,0,0,0,0,0


In [61]:
b311_gb2_backin.shape

(132296, 65)

## Now can import the file (GB311bigtest(reduced)) and join to model dataframe